In [1]:
# SoftBank Stock data from Stooq.com 
# reference https://seanmemo.com/234/

from pandas_datareader import data
start = '2023-04-01'
end = '2023-04-18'
# KONAMI Group Co.
df = data.DataReader('9766.JP', 'stooq', start, end)


In [2]:
print("KONAMI Group Co.")
df

KONAMI Group Co.


,Open,High,Low,Close,Volume
Date,,,,,
2023-04-18,6450,6500,6400,6490,348000
2023-04-17,6480,6490,6390,6410,390500
2023-04-14,6520,6520,6370,6450,655300
2023-04-13,6430,6460,6350,6430,402600
2023-04-12,6420,6480,6390,6440,414000
2023-04-11,6410,6450,6310,6400,515300
2023-04-10,6330,6440,6310,6370,451600
2023-04-07,6400,6400,6260,6290,379400
2023-04-06,6360,6460,6350,6410,624900
